In [19]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import nltk
import re
import keras
from keras import Model

# out libraries
import Utils.W2V as wor2vec
from Utils.classifier import create_Trumpifier
from Utils.genetic_algorithm import GeneticAlgorithm

from keras.callbacks import ModelCheckpoint
# random seed
rseed = 12

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/pablo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [14]:
# load raw data
notTrump_df = pd.read_csv('processedData/processedDataNotDonaldTrump.csv',delimiter="|", header=None)
Trump_df = pd.read_csv('processedData/processedDataDonaldTrump.csv',delimiter="|", header=None)

# delete special characters, generate labels and training data for word2vec
X_t = ["".join(re.split('[`\-=~!@#$%^&*()_+\[\]{};\'\\:"|<,./<>?]', t)).split() for t in Trump_df[1].values.tolist()]
y_t = np.zeros(len(X_t), dtype=int)
X_nt = ["".join(re.split('[`\-=~!@#$%^&*()_+\[\]{};\'\\:"|<,./<>?]', t)).split() for t in notTrump_df[1].values.tolist()]
y_nt = np.ones(len(X_nt), dtype=int)

# make random combinations of each tweet
X_random_t = []
X_random_nt = []

for i in range(len(X_t)):
    row = X_t[i].copy()
    np.random.shuffle(row)
    X_random_t.append(row)

for i in range(len(X_nt)):
    row = X_nt[i].copy()
    np.random.shuffle(row)
    X_random_nt.append(row)
    
y_random_t = np.ones(len(X_random_t), dtype=int) * 2
y_random_nt = np.ones(len(X_random_nt), dtype=int) * 2


# Convert all data to vectors
vec_size = 100
tweet_len = 70

w2v_ds = np.concatenate((X_t, X_nt, X_random_t, X_random_nt))

USE_NO_TWEET_W = False

w2v = wor2vec.W2V(out_size = vec_size, fname="Final_w2v_only_tweets")
# w2v.create_w2v(w2v_ds, USE_NO_TWEET_W)
w2v.load_w2v()
w2v_dict = w2v.dictionary

if USE_NO_TWEET_W:
    # create random data including words that are not inthe tweets
    new_random_X = np.random.choice(list(w2v_dict.keys()), (len(X_t), tweet_len))
    offsets = np.random.randint(1, tweet_len, len(new_random_X))
    new_random_X = [new_random_X[i][:offsets[i]] for i in range(len(new_random_X))]
    new_random_X = [list(c) for c in new_random_X]
    new_random_y = np.ones(len(new_random_X), dtype=int) * 2

    # merge all data
    X = np.concatenate((X_t, X_nt, X_random_t, X_random_nt, new_random_X))
    y = np.concatenate((y_t, y_nt, y_random_t, y_random_nt, new_random_y))
    y = keras.utils.to_categorical(y)
    
else:
    # merge all data
    X = np.concatenate((X_t, X_nt, X_random_t, X_random_nt))
    y = np.concatenate((y_t, y_nt, y_random_t, y_random_nt))
    y = keras.utils.to_categorical(y)
    

X_vec = w2v.vectorize_words(X, tweet_len)

2018-06-20 11:58:58,604 : INFO : loading Word2Vec object from ./Utils/w2v_models/Final_w2v_only_tweets
2018-06-20 11:58:59,340 : INFO : loading wv recursively from ./Utils/w2v_models/Final_w2v_only_tweets.wv.* with mmap=None
2018-06-20 11:58:59,341 : INFO : setting ignored attribute vectors_norm to None
2018-06-20 11:58:59,342 : INFO : loading vocabulary recursively from ./Utils/w2v_models/Final_w2v_only_tweets.vocabulary.* with mmap=None
2018-06-20 11:58:59,344 : INFO : loading trainables recursively from ./Utils/w2v_models/Final_w2v_only_tweets.trainables.* with mmap=None
2018-06-20 11:58:59,345 : INFO : setting ignored attribute cum_table to None
2018-06-20 11:58:59,348 : INFO : loaded ./Utils/w2v_models/Final_w2v_only_tweets
2018-06-20 11:58:59,594 : INFO : saving Word2Vec object under ./Utils/w2v_models/Final_w2v_only_tweets, separately None
2018-06-20 11:58:59,595 : INFO : not storing attribute vectors_norm
2018-06-20 11:58:59,596 : INFO : not storing attribute cum_table
2018-06-

Length input = 171044
Length output = 171044
Unknown words = 0


In [15]:
# # split train and test
# X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.20, shuffle=True, random_state=rseed)

# X_train = np.array([np.array(x) for x in X_train])
# X_test = np.array([np.array(x) for x in X_test])

In [16]:
# classifier = create_Trumpifier(in_size = (tweet_len, vec_size), out_size = len(y[0]))

In [17]:
# Trumpifier_name = "Trumpifier_onlytweets.h5"
# Trumpifier_name = "First_trumpifier.h5"
Trumpifier_name = "Final_only_tweets_trumpifier.h5"

classifier = keras.models.load_model(os.path.join("./Utils/w2v_models/", Trumpifier_name))

In [6]:
# classifier.evaluate(X_test, y_test, batch_size=512, verbose=1)

In [7]:
# # Fit the model
# checkpoint = ModelCheckpoint(os.path.join("./Utils/w2v_models/", Trumpifier_name), 
#                              monitor='val_loss', 
#                              verbose=1, 
#                              save_best_only=True)

# classifier.fit(X_train, y_train, 
#                validation_split = 0.2,
#                epochs = 100, 
#                batch_size = 1024,
#                callbacks = [checkpoint]
#               )

In [8]:
# classifier.evaluate(X_test, y_test, batch_size=32, verbose=1)

In [9]:
import time
t = ["fake", "news"]

ite = 50
GA = GeneticAlgorithm(topics = t, pop_size = 100, n_iter = ite, fitness_model = classifier, 
                      vocab = w2v_dict, cross_rate = 0.1, mut_rate = 0.01, 
                      ind_shape = (tweet_len, vec_size), 
                      evaluationPercentage = [0.2, 0.2, 0.2, 0.2, 0.2])

final_tweet, max_fit_hist, avg_fit_hist, max_len_hist, min_len_hist, avg_len_hist = GA.start_algorithm()


2018-06-20 11:55:45,132 : INFO : loading Word2Vec object from ./Utils/w2v_models/Final_w2v_only_tweets
2018-06-20 11:55:46,477 : INFO : loading wv recursively from ./Utils/w2v_models/Final_w2v_only_tweets.wv.* with mmap=None
2018-06-20 11:55:46,478 : INFO : setting ignored attribute vectors_norm to None
2018-06-20 11:55:46,479 : INFO : loading vocabulary recursively from ./Utils/w2v_models/Final_w2v_only_tweets.vocabulary.* with mmap=None
2018-06-20 11:55:46,480 : INFO : loading trainables recursively from ./Utils/w2v_models/Final_w2v_only_tweets.trainables.* with mmap=None
2018-06-20 11:55:46,480 : INFO : setting ignored attribute cum_table to None
2018-06-20 11:55:46,482 : INFO : loaded ./Utils/w2v_models/Final_w2v_only_tweets
2018-06-20 11:55:46,738 : INFO : saving Word2Vec object under ./Utils/w2v_models/Final_w2v_only_tweets, separately None
2018-06-20 11:55:46,739 : INFO : not storing attribute vectors_norm
2018-06-20 11:55:46,740 : INFO : not storing attribute cum_table
2018-06-

Genetic algorithm with:
	- population size: 100
	- crossover rate: 0.1
	- mutation rate: 0.01



LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger')
  [0m
  Searched in:
    - '/home/pablo/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/home/pablo/anaconda3/envs/MLiP/nltk_data'
    - '/home/pablo/anaconda3/envs/MLiP/share/nltk_data'
    - '/home/pablo/anaconda3/envs/MLiP/lib/nltk_data'
**********************************************************************


In [ ]:
print(final_tweet)

import matplotlib.pyplot as plt

line1, = plt.plot(max_fit_hist, label='Max fitness')
line2, = plt.plot(avg_fit_hist, label='Average fitness')
plt.legend(handles=[line1, line2])
plt.xlabel("Iteration")
plt.ylabel("Fitness")
plt.show()


line1, = plt.plot(max_len_hist, label='Max length')
line2, = plt.plot(min_len_hist, label='Min length')
line3, = plt.plot(avg_len_hist, label='Average length')
plt.legend(handles=[line1, line2, line3])
plt.xlabel("Iteration")
plt.ylabel("Length")
plt.show()

In [ ]:
import csv
directoryDT = './processedData/TestDonaldTrump_tweets.csv'

DTdata_pd = pd.read_csv(directoryDT, delimiter='|')

a = np.array(DTdata_pd.values.tolist())
b = list(a[:,1])

# delete special characters, generate labels and training data for word2vec
X_t = ["".join(re.split('[`\-=~!@#$%^&*()_+\[\]{};\'\\:"|<,./<>?]', t)).split() for t in b]

population_words = X_t[10:11]+ X_t[1:2]
pop_size = len(population_words)
ind_shape = (70, 100)
vocab = w2v_dict

population_init = [[( iw , vocab[iw][0]) for iw in ind_words] for ind_words in population_words]


for i in range(len(population_init)):                
    #add zero tuples at the end
    cut_offset = len(population_init[i])
    for j in range(ind_shape[0] - cut_offset):
        population_init[i].append(  ('' , np.zeros(ind_shape[1], dtype = np.float32) ) )
popu = population_init


In [ ]:
tweets = []
for i in popu:
    tweets.append(' '.join( np.array([x for x,_ in i])) )
    
print(tweets)
a = GA.evaluate_population(popu)
print(a)